In [96]:
using MacroTools
using MacroTools: striplines
using JSON

In [1]:
using HTTP
using JSON

In [ ]:
macro rest_str(doc)

In [2]:
#From https://developer.github.com/v3/search/
doc = """
Find repositories via various criteria. This method returns up to 100 results per page.

GET /search/repositories

Parameters
Name 	Type 	Description
q 	string 	Required. The search keywords, as well as any qualifiers.
sort 	string 	The sort field. One of stars, forks, or updated. Default: results are sorted by best match.
order 	string 	The sort order if sort parameter is provided. One of asc or desc. Default: desc
"""

"Find commits via various criteria. This method returns up to 100 results per page.\n\nGET /search/commits\n\nConsiderations for commit search\n\nOnly the default branch is considered. In most cases, this will be the master branch.\nParameters\nName \tType \tDescription\nq \tstring \tRequired. The search terms.\nsort \tstring \tThe sort field. Can be author-date or committer-date. Default: results are sorted by best match.\norder \tstring \tThe sort order if sort parameter is provided. One of asc or desc. Default: desc\n"

In [111]:
function declare_api(root, method, endpoint, param_names)
    function_name = Symbol(join(split(endpoint, "/"; keep=false), "_"))
    param_sig = Expr(:parameters,Expr.(:kw, Symbol.(param_names), :nothing)...)
    
    set_query_code = Expr(:block)
    set_query_code.args = map(param_names) do param
        quote
             if $(Symbol(param)) != nothing
                query[$(String(param))] = $(Symbol(param))
            end
        end |> MacroTools.unblock 
    end
     
    quote
        function $(function_name)($(param_sig))
            query = Dict()
            $(set_query_code)
            uri = HTTP.URI($root)
            uri = merge(uri; path=uri.path*$endpoint, query=query)
            resp = HTTP.request($method, uri, ["USER-AGENT"=>"RESTful.jl"])
            JSON.parse(String(resp.body))
        end
    end|> MacroTools.unblock
end
    
declare_api("https://api.github.com", "GET", "/search/repositories", ["q", "sort", "order"])

:(function search_repositories(; q=nothing, sort=nothing, order=nothing) # In[111], line 16:
        query = Dict() # In[111], line 17:
        begin 
            if q != nothing # In[111], line 9:
                query["q"] = q
            end
            if sort != nothing # In[111], line 9:
                query["sort"] = sort
            end
            if order != nothing # In[111], line 9:
                query["order"] = order
            end
        end # In[111], line 18:
        uri = HTTP.URI("https://api.github.com") # In[111], line 19:
        uri = merge(uri; path=uri.path * "/search/repositories", query=query) # In[111], line 20:
        resp = HTTP.request("GET", uri, ["USER-AGENT" => "RESTful.jl"]) # In[111], line 21:
        JSON.parse(String(resp.body))
    end)

In [112]:
eval(declare_api("https://api.github.com", "GET", "/search/repositories", ["q", "sort", "order"]))

search_repositories (generic function with 1 method)

In [113]:
x=search_repositories(q="julia")

Dict{String,Any} with 3 entries:
  "incomplete_results" => false
  "items"              => Any[Dict{String,Any}(Pair{String,Any}("pulls_url", "h…
  "total_count"        => 12141

In [127]:
[itm["name"] for itm in x["items"]]

30-element Array{String,1}:
 "julia"                   
 "IJulia.jl"               
 "Julia.jl"                
 "METADATA.jl"             
 "Mocha.jl"                
 "julialang.github.com"    
 "JuliaBox"                
 "Gadfly.jl"               
 "JuliaByExample"          
 "Optim.jl"                
 "QuantEcon.jl"            
 "learn-julia-the-hard-way"
 "JuliaCourseNotebooks"    
 ⋮                         
 "julia-vim"               
 "julia-studio"            
 "julia-mit"               
 "Images.jl"               
 "julia-tutorial"          
 "homebrew-julia"          
 "DataStructures.jl"       
 "Documenter.jl"           
 "Winston.jl"              
 "Escher.jl"               
 "JuliaFEM.jl"             
 "Graphs.jl"               

# TODO:

 - Parse key description features out of documentation, via pattern matching. (most doc seem fairly consitant)
 - Allow use to specify if the returned result is e.g. JSON or XML or what ever
 - Handle standard(?) pagination stuff
 - Handle auth stuff (prob means just letting user add stuff to the request?)
